import the necessary packages

In [ ]:
import numpy as np
import argparse
import imutils
import cv2

construct the argument parse and parse the arguments

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to deep learning segmentation model")
ap.add_argument("-c", "--classes", required=True,
	help="path to .txt file containing class labels")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-l", "--colors", type=str,
	help="path to .txt file containing colors for labels")
ap.add_argument("-w", "--width", type=int, default=500,
	help="desired width (in pixels) of input image")
args = vars(ap.parse_args())

load the class label names

In [ ]:
CLASSES = open(args["classes"]).read().strip().split("\n")

if a colors file was supplied, load it from disk

In [ ]:
if args["colors"]:
	COLORS = open(args["colors"]).read().strip().split("\n")
	COLORS = [np.array(c.split(",")).astype("int") for c in COLORS]
	COLORS = np.array(COLORS, dtype="uint8")

otherwise, we need to randomly generate RGB colors for each class<br>
label

In [ ]:
else:
	# initialize a list of colors to represent each class label in
	# the mask
	np.random.seed(42)
	COLORS = np.random.randint(0, 255, size=(len(CLASSES) - 1, 3),
		dtype="uint8")
	COLORS = np.vstack([[0, 0, 0], COLORS]).astype("uint8")

initialize the legend visualization

In [ ]:
legend = np.zeros(((len(CLASSES) * 25) + 25, 300, 3), dtype="uint8")

loop over the class names + colors

In [ ]:
for (i, (className, color)) in enumerate(zip(CLASSES, COLORS)):
	# draw the class name + color on the legend
	color = [int(c) for c in color]
	cv2.putText(legend, className, (5, (i * 25) + 17),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
	cv2.rectangle(legend, (100, (i * 25)), (300, (i * 25) + 25),
		tuple(color), -1)

load our serialized model from disk

In [ ]:
net = cv2.dnn.readNet(args["model"])

load the input image, resize it, and construct a blob from it,<br>
by keeping mind that the original input image dimensions<br>
ENet was trained on was 1024x512

In [ ]:
image = cv2.imread(args["image"])
image = imutils.resize(image, width=args["width"])
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (1024, 512), 0,
	swapRB=True, crop=False)

perform a forward pass using the segmentation model

In [ ]:
net.setInput(blob)

In [ ]:
output = net.forward()

infer the total number of classes along with the spatial dimensions<br>
of the mask image via the shape of the output array

In [ ]:
(numClasses, height, width) = output.shape[1:4]

our output class ID map will be num_classes x height x width in<br>
size, so we take the argmax to find the class label with the<br>
largest probability for each and every (x, y)-coordinate in the<br>
image

In [ ]:
classMap = np.argmax(output[0], axis=0)

given the class ID map, we can map each of the class IDs to its<br>
corresponding color

In [ ]:
mask = COLORS[classMap]

resize the mask such that its dimensions match the<br>
original size of the input image 

In [ ]:
mask = cv2.resize(mask, (image.shape[1], image.shape[0]),
	interpolation=cv2.INTER_NEAREST)

perform a weighted combination of the input image with the mask to<br>
form an output visualization

In [ ]:
output = ((0.4 * image) + (0.6 * mask)).astype("uint8")

show the input and output images

In [ ]:
cv2.imshow("Legend", legend)
cv2.imshow("Input", image)
cv2.imshow("Output", output)
cv2.waitKey(0)

USAGE<br>
python segment.py --model enet-cityscapes/enet-model.net --classes enet-cityscapes/enet-classes.txt --colors enet-cityscapes/enet-colors.txt --image images/example_01.png